<a href="https://colab.research.google.com/github/subramanya4shenoy/MachineLearningNbs/blob/main/PS_S04E12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# prompt: connect to a kaggle problem

!pip install kaggle

# Upload your kaggle.json file (contains API key and secret)
from google.colab import files
files.upload()

# Create the .kaggle directory and move the file
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

# Set permissions for the file
!chmod 600 ~/kaggle.json

# Download the dataset (replace with the actual competition name and dataset)
!kaggle competitions download -c playground-series-s4e12 # example competition name

# Unzip the downloaded dataset
!unzip playground-series-s4e12.zip # example dataset file name

Saving kaggle.json to kaggle.json
chmod: cannot access '/root/kaggle.json': No such file or directory
100% 75.5M/75.5M [00:03<00:00, 32.9MB/s]
100% 75.5M/75.5M [00:03<00:00, 24.5MB/s]
Archive:  playground-series-s4e12.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               




---

# EDA : Analysing data

---


*   From given data we need to identify the premium amount
*   1 column to predict.

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv('train.csv', index_col='id')

In [6]:
data.head()

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
id,,,,,,,,,,,,,,,,,,,,
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [7]:
data.columns

Index(['Age', 'Gender', 'Annual Income', 'Marital Status',
       'Number of Dependents', 'Education Level', 'Occupation', 'Health Score',
       'Location', 'Policy Type', 'Previous Claims', 'Vehicle Age',
       'Credit Score', 'Insurance Duration', 'Policy Start Date',
       'Customer Feedback', 'Smoking Status', 'Exercise Frequency',
       'Property Type', 'Premium Amount'],
      dtype='object')

In [8]:
data.dtypes

,0
Age,float64
Gender,object
Annual Income,float64
Marital Status,object
Number of Dependents,float64
Education Level,object
Occupation,object
Health Score,float64
Location,object
Policy Type,object



### Segrgating
* Categorical Columns
* Numerical Columns



In [9]:
# Identify categorical and numerical columns
categorical_cols = data.select_dtypes(include=['object']).columns.tolist()
numerical_cols = data.select_dtypes(include=['number']).columns.tolist()

## **Categorical Columns**: Check for how many uniq values in categorical columns


*  To decide on Onehotencoding or label encoding



In [10]:
for col in categorical_cols:
    unique_values = data[col].nunique()
    null_values = data[col].isnull().sum()
    print(f"'{col}' - {unique_values} - {null_values}")

'Gender' - 2 - 0
'Marital Status' - 3 - 18529
'Education Level' - 4 - 0
'Occupation' - 3 - 358075
'Location' - 3 - 0
'Policy Type' - 3 - 0
'Policy Start Date' - 167381 - 0
'Customer Feedback' - 3 - 77824
'Smoking Status' - 2 - 0
'Exercise Frequency' - 4 - 0
'Property Type' - 3 - 0


## Observation


*   'Gender', 'Smoking Status' - onehot encoding 🟢
*   'Education Level', 'Property Type' 'Exercise Frequency' 'Policy Type', 'Location', - Label encoding 🟢
* 'Marital Status', 'Occupation', 'Policy Start Date', 'Customer Feedback' - Needs to look at further




## Handling 'Marital Status'


In [11]:
# making a copy of data to play around with maretial status
data_copy = data.copy()

In [12]:
data_copy['Marital Status'].value_counts()

,count
Marital Status,
Single,395391
Married,394316
Divorced,391764


### Number of missing values

In [13]:
data_copy['Marital Status'].isnull().sum()

18529

### TRying to logically fill the missing values

* From corresponding 'Dependant' column - 0 dependant ~ 'Single'
* From corresponding 'Age' column. I see that its eually distributed accross the all age groups so i am gonna remove all null marital status.



In [14]:
# prompt: for all missing values in data['Marital Status'], show me the corresponding Dependant column

# Display rows where 'Marital Status' is missing along with the 'Dependant' column
missing_marital_status = data_copy[data_copy['Marital Status'].isnull()]
missing_marital_status[['Number of Dependents']].value_counts()

,count
Number of Dependents,
4.0,3403
3.0,3399
0.0,3384
1.0,3255
2.0,3216




*   Number of Depents = 0 can be treated as single.


In [15]:
# Fill missing 'Marital Status' based on 'Number of Dependents'
data_copy.loc[(data_copy['Marital Status'].isnull()) & (data_copy['Number of Dependents'] == 0), 'Marital Status'] = 'Single'

### Checking again the count of missing values

In [16]:
data_copy['Marital Status'].isnull().sum()

15145

In [17]:
# prompt: from data_copy group the data into 6 groups starting with age 18 till 64 each section should cover next 8 ages. show me the count of people int those sections

# Group data into age bins
bins = range(18, 67, 8)  # Create bins from 18 to 64, with a step of 8
labels = [f'{i}-{i+7}' for i in bins[:-1]] # Create labels for each group

data_copy['Age_Group'] = pd.cut(data_copy['Age'], bins=bins, labels=labels, right=False)

# Count people in each age group
age_group_counts = data_copy['Marital Status'].isnull().groupby(data_copy['Age_Group']).value_counts()

# Display the
age_group_counts

<ipython-input-17-fed066cfd77c>:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_group_counts = data_copy['Marital Status'].isnull().groupby(data_copy['Age_Group']).value_counts()


Age_Group  Marital Status
18-25      False             194157
           True                2459
26-33      False             196946
           True                2545
34-41      False             199328
           True                2495
42-49      False             200068
           True                2673
50-57      False             200426
           True                2548
58-65      False             175478
           True                2172
Name: count, dtype: int64

In [18]:
# prompt: remove all row with null and missing Marital Status values from the data_copy

# Remove rows with null or missing 'Marital Status' values
data_copy.dropna(subset=['Marital Status'], inplace=True)

In [19]:
data_copy['Marital Status'].value_counts()

,count
Marital Status,
Single,398775
Married,394316
Divorced,391764


In [20]:
data_copy['Marital Status'].isnull().sum()

0

In [21]:
data_copy.shape

(1184855, 21)

#### *⬆* Marital Status is good for Label classification now. 🟢

## Handling Occupation column

In [22]:
data_copy = data.copy()

In [23]:
data_copy.columns

Index(['Age', 'Gender', 'Annual Income', 'Marital Status',
       'Number of Dependents', 'Education Level', 'Occupation', 'Health Score',
       'Location', 'Policy Type', 'Previous Claims', 'Vehicle Age',
       'Credit Score', 'Insurance Duration', 'Policy Start Date',
       'Customer Feedback', 'Smoking Status', 'Exercise Frequency',
       'Property Type', 'Premium Amount'],
      dtype='object')

In [24]:
data_copy['Occupation'].value_counts()

,count
Occupation,
Employed,282750
Self-Employed,282645
Unemployed,276530


In [25]:
data_copy['Occupation'].isnull().sum()

358075

### Trying to fill missing the value using the annual income columns

* Removing whose 'Anual income is missing' and 'Occupation' is missing
*  if there is a anual income and the employment is less than mean of unemployed columns mean then we classify the occupation as 'unemployed' and other wise its 'self emplyed'. if the 'Education Level' is Low then unemployed, we take both consideration and see how it will reduce.



In [26]:
# prompt: remove rows whos anual income is missing and occupation is missing

# Remove rows where both 'Annual Income' and 'Occupation' are missing
data_copy.dropna(subset=['Annual Income', 'Occupation'], inplace=True)

In [27]:
data_copy['Occupation'].isnull().sum()

0

#### *⬆* Occupation is good for Label classification now. 🟢

## Handling 'Policy Start date' column

In [28]:
data_copy = data.copy()

In [29]:
data_copy.head()

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
id,,,,,,,,,,,,,,,,,,,,
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0




*   Check for missing values
*   convert date into days since the policy started



In [30]:
data_copy['Policy Start Date'].isnull().sum()

0

In [31]:
from datetime import datetime

# Convert 'Policy Start Date' to datetime objects
data_copy['Policy Start Date'] = pd.to_datetime(data_copy['Policy Start Date'])

# Calculate the difference between today's date and the policy start datew
today = datetime.now()
data_copy['Policy Days'] = (today - data_copy['Policy Start Date']).dt.days

# Drop the original 'Policy Start Date' column
data_copy = data_copy.drop('Policy Start Date', axis=1)

data_copy.head()

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount,Policy Days
id,,,,,,,,,,,,,,,,,,,,
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,Poor,No,Weekly,House,2869.0,370
1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,Average,Yes,Monthly,House,1483.0,564
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,Good,Yes,Weekly,House,567.0,454
3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,Poor,Yes,Daily,Apartment,765.0,198
4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,Poor,Yes,Weekly,House,2022.0,1122


### Good for scaler encoding

## Handling 'Customer Feedback' column

In [32]:
data_copy = data.copy()

In [33]:
data_copy.head()

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
id,,,,,,,,,,,,,,,,,,,,
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [34]:
data_copy['Customer Feedback'].value_counts()

,count
Customer Feedback,
Average,377905
Poor,375518
Good,368753


In [35]:
data_copy['Customer Feedback'].isnull().sum()

77824

In [36]:
## checking if i remove how much data we need to remove.
(data_copy['Customer Feedback'].isnull().sum() / data_copy.shape[0]) * 100

6.485333333333333

In [37]:
# Drop rows with null values in the 'Customer Feedback' column
data_copy.dropna(subset=['Customer Feedback'], inplace=True)

In [38]:
data_copy.shape

(1122176, 20)

### 'Customer Feedback' Columns is good for Label classification



---



## **Numerical Columns**: Check for how many uniq values in Numerical columns

### Observation

* 'Age',
 'Annual Income',
 'Number of Dependents',
 'Health Score',
 'Previous Claims',
 'Vehicle Age',
 'Credit Score',
 'Insurance Duration',
 'Premium Amount' all numerical
*   Removing missing values from 'Insurance Duration' and 'Vehicle Age' which is only 8 7 rows
* Removing 'Age' as well
* for missing anual incomde, we will identify a another row which matches similar age, occupation, gender, education level and find the mean of that then will fill with that
* Dropping rest of the annual income missing column as its only 1% of the data
*Handling missing health score, with other columns values, age, gender, 'Smoking Status', 'Exercise Frequency'
* Number of dependants I am going to fill it with 2
* Credit score i will be filling it with matching row, and what ever is left (3% of data) dropping them
* for missing Previous claim using the matching location, polici type's mode of Previous Claims

In [39]:
data_copy = data.copy()

In [40]:
data_copy[numerical_cols].isnull().sum().sort_values(ascending=True)

,0
Premium Amount,0
Insurance Duration,1
Vehicle Age,6
Age,18705
Annual Income,44949
Health Score,74076
Number of Dependents,109672
Credit Score,137882
Previous Claims,364029


In [41]:
data_copy.dropna(subset='Vehicle Age', inplace=True);
data_copy.dropna(subset='Insurance Duration', inplace=True);
data_copy.dropna(subset='Age', inplace=True);

In [42]:
data_copy[numerical_cols].isnull().sum().sort_values(ascending=True)

,0
Age,0
Vehicle Age,0
Insurance Duration,0
Premium Amount,0
Annual Income,44208
Health Score,72970
Number of Dependents,107930
Credit Score,135733
Previous Claims,358282


In [43]:
data_copy.columns

Index(['Age', 'Gender', 'Annual Income', 'Marital Status',
       'Number of Dependents', 'Education Level', 'Occupation', 'Health Score',
       'Location', 'Policy Type', 'Previous Claims', 'Vehicle Age',
       'Credit Score', 'Insurance Duration', 'Policy Start Date',
       'Customer Feedback', 'Smoking Status', 'Exercise Frequency',
       'Property Type', 'Premium Amount'],
      dtype='object')

In [44]:
# Group data by relevant columns and calculate the mean annual income for each group
grouped = data_copy.groupby(['Age', 'Occupation', 'Gender', 'Education Level'])['Annual Income'].mean()

# Function to fill missing annual income based on group means
def fill_annual_income(row):
    if pd.isnull(row['Annual Income']):
        try:
            return grouped[row['Age'], row['Occupation'], row['Gender'], row['Education Level']]
        except KeyError:
            return row['Annual Income']  # Keep NaN if no matching group found
    else:
        return row['Annual Income']

# Apply the function to fill missing values
data_copy['Annual Income'] = data_copy.apply(fill_annual_income, axis=1)

In [45]:
data_copy[numerical_cols].isnull().sum().sort_values(ascending=True)

,0
Age,0
Vehicle Age,0
Insurance Duration,0
Premium Amount,0
Annual Income,12998
Health Score,72970
Number of Dependents,107930
Credit Score,135733
Previous Claims,358282


In [46]:
(data_copy['Annual Income'].isnull().sum()/data_copy.shape[0]) * 100

1.1003243916809449

In [47]:
# dropping missing anual income rows as this is just 1% of data
data_copy.dropna(subset='Annual Income', inplace=True);

In [48]:
data_copy[numerical_cols].isnull().sum().sort_values(ascending=True)

,0
Age,0
Annual Income,0
Vehicle Age,0
Insurance Duration,0
Premium Amount,0
Health Score,72335
Number of Dependents,106724
Credit Score,134777
Previous Claims,354465


In [49]:
data_copy['Health Score'].describe()

,Health Score
count,1.095955e+06
mean,2.560254e+01
std,1.219400e+01
min,2.012237e+00
25%,1.591801e+01
50%,2.457222e+01
75%,3.449783e+01
max,5.897591e+01


In [50]:
# prompt: Handling missing health score, with other columns values, age, gender, 'Smoking Status', 'Exercise Frequency'

# Group data by relevant columns and calculate the mean health score for each group
grouped_health = data_copy.groupby(['Age', 'Gender', 'Smoking Status', 'Exercise Frequency'])['Health Score'].mean()

# Function to fill missing health score based on group means
def fill_health_score(row):
    if pd.isnull(row['Health Score']):
        try:
            return grouped_health[row['Age'], row['Gender'], row['Smoking Status'], row['Exercise Frequency']]
        except KeyError:
            return row['Health Score']  # Keep NaN if no matching group found
    else:
        return row['Health Score']

# Apply the function to fill missing values
data_copy['Health Score'] = data_copy.apply(fill_health_score, axis=1)

# Check for remaining missing values in 'Health Score'
print(data_copy['Health Score'].isnull().sum())

0


In [51]:
data_copy[numerical_cols].isnull().sum().sort_values(ascending=True)

,0
Age,0
Annual Income,0
Health Score,0
Vehicle Age,0
Insurance Duration,0
Premium Amount,0
Number of Dependents,106724
Credit Score,134777
Previous Claims,354465


In [52]:
data_copy.head()

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
id,,,,,,,,,,,,,,,,,,,,
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [53]:
# prompt: Number of dependants I am going to fill it with 2

# Fill missing 'Number of Dependents' with 2
data_copy['Number of Dependents'].fillna(2, inplace=True)

<ipython-input-53-3ba9d7fc6ae8>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_copy['Number of Dependents'].fillna(2, inplace=True)


In [54]:
data_copy[numerical_cols].isnull().sum().sort_values(ascending=True)

,0
Age,0
Annual Income,0
Number of Dependents,0
Health Score,0
Vehicle Age,0
Insurance Duration,0
Premium Amount,0
Credit Score,134777
Previous Claims,354465


In [55]:
# prompt: fill the credit score if there exist a matching row

# Group data by relevant columns and calculate the mean credit score for each group
grouped_credit = data_copy.groupby(['Age', 'Occupation', 'Gender', 'Education Level'])['Credit Score'].mean()

# Function to fill missing credit score based on group means
def fill_credit_score(row):
    if pd.isnull(row['Credit Score']):
        try:
            return grouped_credit[row['Age'], row['Occupation'], row['Gender'], row['Education Level']]
        except KeyError:
            return row['Credit Score']  # Keep NaN if no matching group found
    else:
        return row['Credit Score']

# Apply the function to fill missing values
data_copy['Credit Score'] = data_copy.apply(fill_credit_score, axis=1)

# Check for remaining missing values in 'Credit Score'
print(data_copy['Credit Score'].isnull().sum())

39182


In [56]:
(data_copy['Credit Score'].isnull().sum()/data_copy.shape[0]) * 100

3.353790582817622

In [57]:
data_copy.dropna(subset='Credit Score', inplace=True);

In [58]:
data_copy[numerical_cols].isnull().sum().sort_values(ascending=True)

,0
Age,0
Annual Income,0
Number of Dependents,0
Health Score,0
Vehicle Age,0
Credit Score,0
Insurance Duration,0
Premium Amount,0
Previous Claims,342604


In [59]:
data_copy.head()

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
id,,,,,,,,,,,,,,,,,,,,
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.000000,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.000000,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,595.088333,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.000000,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.000000,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [60]:
data_copy['Previous Claims'].value_counts()

,count
Previous Claims,
0.0,287674
1.0,283406
2.0,157373
3.0,45862
4.0,9941
5.0,1904
6.0,281
7.0,55
8.0,8


In [61]:
# prompt: for missing Previous claim using the matching location, polici type's mode of Previous Claims

# Group data by 'Location' and 'Policy Type' and calculate the mode of 'Previous Claims'
grouped_claims = data_copy.groupby(['Location', 'Policy Type'])['Previous Claims'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)

# Function to fill missing 'Previous Claims' based on group modes
def fill_previous_claims(row):
    if pd.isnull(row['Previous Claims']):
        try:
            return grouped_claims[row['Location'], row['Policy Type']]
        except KeyError:
            return row['Previous Claims']  # Keep NaN if no matching group found
    else:
        return row['Previous Claims']

# Apply the function to fill missing values
data_copy['Previous Claims'] = data_copy.apply(fill_previous_claims, axis=1)

# Check for remaining missing values in 'Previous Claims'
print(data_copy['Previous Claims'].isnull().sum())

0


In [62]:
data_copy[numerical_cols].isnull().sum().sort_values(ascending=True)

,0
Age,0
Annual Income,0
Number of Dependents,0
Health Score,0
Previous Claims,0
Vehicle Age,0
Credit Score,0
Insurance Duration,0
Premium Amount,0


### ⬆ All numerical columns are handled



---



# Pre processing methods

In [169]:
data_copy = data.copy()

### Encoding

In [103]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler

#### Special processing of 'Marital Status', 'Occupation', 'Policy Start Date', 'Customer Feedback'

In [150]:
def handle_marital_status(data):
    # Handle missing values in 'Marital Status'
    data.loc[(data['Marital Status'].isnull()) & (data['Number of Dependents'] == 0), 'Marital Status'] = 'Single'
    data.dropna(subset=['Marital Status'], inplace=True)
    return data

In [115]:
def handle_occupation(data):
    # Handle missing values in 'Occupation'
    data.dropna(subset=['Occupation'], inplace=True)
    return data

In [116]:
from datetime import datetime

def handle_policy_start_date(data):
    # Convert 'Policy Start Date' to datetime objects
    data['Policy Start Date'] = pd.to_datetime(data['Policy Start Date'])

    # Calculate the difference between today's date and the policy start datew
    today = datetime.now()
    data['Policy Days'] = (today - data['Policy Start Date']).dt.days

    # Drop the original 'Policy Start Date' column
    data = data.drop('Policy Start Date', axis=1)
    return data

In [117]:
def handle_customer_feedback(data):
    # Drop rows with null values in the 'Customer Feedback' column
    data.dropna(subset=['Customer Feedback'], inplace=True)
    return data

In [168]:
# categorical column enoding
def categorical_cols_encoding(data):

    # OneHotEncoding
    one_hot_encoder_cols = ['Gender', 'Smoking Status']
    one_hot_encoder_gender = OneHotEncoder()
    gender_encoded = one_hot_encoder_gender.fit_transform(data[['Gender']]).toarray()
    gender_encoded_df = pd.DataFrame(gender_encoded, columns = one_hot_encoder_gender.get_feature_names_out(['Gender']))

    one_hot_encoder_smoking = OneHotEncoder()
    smoking_encoded = one_hot_encoder_smoking.fit_transform(data[['Smoking Status']]).toarray()
    smoking_encoded_df = pd.DataFrame(smoking_encoded, columns = one_hot_encoder_smoking.get_feature_names_out(['Smoking Status']))

    # data = pd.concat([data, gender_encoded_df, smoking_encoded_df], axis=1)
    data.drop(columns=['Gender', 'Smoking Status'], inplace=True)

    #Label calssification
    label_encoder_cols = ['Education Level', 'Property Type', 'Exercise Frequency', 'Policy Type', 'Location'];
    label_encoder = LabelEncoder()
    for col in label_encoder_cols:
        data[col] = label_encoder.fit_transform(data[col])
        data = pd.concat([data, pd.get_dummies(data[col], prefix=col)], axis=1)
        data.drop(columns=[col], inplace=True)

    return data

In [119]:
# numeric column encoding
def numerical_cols_encoding(data):
    numerical_cols = data.select_dtypes(include=['number']).columns.tolist()

    # scaler = StandardScaler()
    # data[numerical_cols] = scaler.fit_transform(data[numerical_cols])
    return data

### Preprocesiing methods

In [145]:
def preprocess_categorical_columns(data):
    #Handle special cases
    data = handle_marital_status(data) ## Marital status
    data = handle_occupation(data)  ## Occupation
    data = handle_policy_start_date(data) ## Policy Start date
    data = handle_customer_feedback(data)  ## Customer Feedback

    data = categorical_cols_encoding(data)
    return data

In [121]:
def preprocess_numerical_columns(data):
    data = numerical_cols_encoding(data)
    return data

In [122]:
def pre_processing(data):
    data = preprocess_categorical_columns(data)
    data = preprocess_numerical_columns(data)
    return data

In [170]:
data_copy = data.copy()
pre_processed_data = pre_processing(data_copy);
pre_processed_data

,Age,Annual Income,Marital Status,Number of Dependents,Occupation,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,...,Exercise Frequency_0,Exercise Frequency_1,Exercise Frequency_2,Exercise Frequency_3,Policy Type_0,Policy Type_1,Policy Type_2,Location_0,Location_1,Location_2
id,,,,,,,,,,,,,,,,,,,,,
0,19.0,10049.0,Married,1.0,Self-Employed,22.598761,2.0,17.0,372.0,5.0,...,False,False,False,True,False,False,True,False,False,True
2,23.0,25602.0,Divorced,3.0,Self-Employed,47.177549,1.0,14.0,NaN,3.0,...,False,False,False,True,False,False,True,False,True,False
4,21.0,39651.0,Single,1.0,Self-Employed,20.376094,0.0,8.0,598.0,4.0,...,False,False,False,True,False,False,True,True,False,False
7,48.0,127237.0,Divorced,2.0,Employed,5.769783,1.0,11.0,398.0,5.0,...,False,False,True,False,False,True,False,False,True,False
9,44.0,52447.0,Married,2.0,Employed,20.473718,1.0,9.0,635.0,3.0,...,True,False,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199991,59.0,23706.0,Divorced,4.0,Self-Employed,24.913204,NaN,17.0,NaN,1.0,...,False,True,False,False,False,True,False,False,True,False
1199992,53.0,6837.0,Married,2.0,Self-Employed,17.844235,NaN,15.0,406.0,4.0,...,False,False,True,False,False,True,False,False,False,True
1199994,34.0,23456.0,Single,4.0,Self-Employed,14.783439,NaN,12.0,548.0,9.0,...,False,True,False,False,True,False,False,True,False,False
